In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from rag.retriever import process_pdf_document, create_vectorstore, rag_retriever

# Function to create embeddings
# def create_embeddings(text_chunks):
#     embeddings = embeddings_model.encode(text_chunks, show_progress_bar=True)
#     return embeddings

# load the document
documents, parent_splitter, child_splitter = process_pdf_document('/Users/misanmeggison/Downloads/DepressionGuide-web.pdf')

# Create the vectorstore
vectorstore, store = create_vectorstore()

# Create the retriever
retriever = rag_retriever(vectorstore, store, documents, parent_splitter, child_splitter)


In [12]:
import os
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from rag.generator import load_llm
from langchain.prompts import PromptTemplate


# Load the LLM model
llm = load_llm()


# get user message prompts
def get_prompts():
    prompt_dir = '/Users/misanmeggison/HyderabadIndiaChapter_MentalHealthWellbeingFomoSocialMedia/src/prompts'
    with open(os.path.join(prompt_dir, 'system_prompt_template.txt'), 'r') as f:
        system_prompt_template = f.read()

    with open(os.path.join(prompt_dir, 'user_prompt_template.txt'), 'r') as f:
        user_message = f.read()

    with open(os.path.join(prompt_dir, 'condense_question_prompt_template.txt'), 'r') as f:
        condense_question_prompt = f.read()
        

    return system_prompt_template, user_message, condense_question_prompt

# get user message prompts
system_prompt_template, user_message, condense_question_prompt = get_prompts()

# create message templates
ANSWER_PROMPT = ChatPromptTemplate.from_template(system_prompt_template)

# 
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_question_prompt)


# user_message = HumanMessagePromptTemplate.from_template(user_message)

# # compile messgaes
# chat_prompt = ChatPromptTemplate.from_messages(
#     [system_message, user_message])

ANSWER_PROMPT



WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/misanmeggison/.cache/huggingface/token
Login successful


ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='[INST] \nYou are a Mental Health Specialist (Counsellor)\nYou are to provide support and guidance to individuals discussing their feelings and mental health.\nYour responses should be warm, understanding, and encourage open communication.\nIf the user asks for factual information, use the context of the converation to provide relevant responses.\nDo not provide professional diagnosis to the user and is unsure be honest and say "I can\'t answer this question right now". \nAlways provide answers to user queries without deviating from the subject matter\n\nContext: {context}\n\nQuestion: {question}\n[/INST] '))])

In [13]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# Instantiate ConversationBufferMemory
memory = ConversationBufferMemory(
 return_messages=True, output_key="answer", input_key="question"
)

In [15]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")
from langchain_core.output_parsers import StrOutputParser


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | llm,
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [16]:
def call_conversational_rag(question, chain, memory):
    """
    Calls a conversational RAG (Retrieval-Augmented Generation) model to generate an answer to a given question.

    This function sends a question to the RAG model, retrieves the answer, and stores the question-answer pair in memory 
    for context in future interactions.

    Parameters:
    question (str): The question to be answered by the RAG model.
    chain (LangChain object): An instance of LangChain which encapsulates the RAG model and its functionality.
    memory (Memory object): An object used for storing the context of the conversation.

    Returns:
    dict: A dictionary containing the generated answer from the RAG model.
    """
    
    # Prepare the input for the RAG model
    inputs = {"question": question}

    # Invoke the RAG model to get an answer
    result = chain.invoke(inputs)
    
    # Save the current question and its answer to memory for future context
    memory.save_context(inputs, {"answer": result["answer"]})
    
    # Return the result
    return result

In [17]:
question = "What is depression?"
call_conversational_rag(question, final_chain, memory)

{'answer': "Based on the information provided in the text you've shared, the causes of depression are believed to be a combination of genetic and environmental factors. Genetic factors, such as having a family history of depression, can make a person more prone to developing depression. Environmental factors, such as stressors like the loss of a loved one, family conflicts, or financial problems, can also contribute to the development of depression. It's important to remember that there is no single cause of depression, and everyone's experience is unique. If you're feeling persistently sad or experiencing a loss of interest in activities, it may be helpful to speak with a mental health professional for further evaluation and support.",
 'docs': [Document(page_content='4       Depression: Parents’ Medication Guide\nIntroduction\nT\nhe original Parents Medical Guide on \ntreating depression was published in \n2005, and a revision was published in \n2010, through collaboration by the Ame

In [21]:
question = "What are the symptoms of depression?"
answer = call_conversational_rag(question, final_chain, memory)

print(answer['answer'])

I'm glad you've been open about discussing the symptoms of depression as outlined in the Diagnostic and Statistical Manual of Mental Disorders (DSM-5). Based on our previous conversations, the specific symptoms of depression include:

1. Depressed, sad, or irritable mood: Feeling sad, down, or irritable most of the time for a minimum of two weeks.
2. Significant loss of interest or pleasure in activities: Losing interest in activities that were once enjoyed and finding little pleasure in daily life.
3. Significant weight loss, weight gain, or appetite changes: Experiencing a noticeable change in weight or appetite that is not due to a medical condition or intentional weight loss/gain.
4. Difficulty falling asleep and/or staying asleep or sleeping too much: Having trouble falling asleep or staying asleep, or sleeping too much and feeling tired upon waking.
5. Restlessness, unable to sit still or being slowed down: Feeling restless and unable to sit still, or feeling slowed down and unab

In [22]:
question = "I've had a really bad day today"
answer = call_conversational_rag(question, final_chain, memory)

print(answer['answer'])

1. I'm here to listen and support you. I understand that you've had a tough day, and it's important to be aware of the symptoms of depression. Have you experienced any of the following symptoms today:

   a. Depressed, sad, or irritable mood
   b. Significant loss of interest or pleasure in activities
   c. Significant weight loss, weight gain, or appetite changes
   d. Difficulty falling asleep and/or staying asleep or sleeping too much
   e. Restlessness, unable to sit still or being slowed down
   f. Fatigue or loss of energy
   g. Feelings of worthlessness or excessive or inappropriate feelings of guilt
   h. Difficulties in concentrating or making decisions
   i. Constant thoughts of death, suicidal thinking, or a suicide attempt?

It's essential to remember that having one or two symptoms does not necessarily mean you have depression. However, if you've been experiencing several of these symptoms for more than two weeks, it might be a good idea to reach out to a mental health pro

In [23]:
print(memory.chat_memory)

Human: What is depression?
AI: Based on the information provided in the text you've shared, the causes of depression are believed to be a combination of genetic and environmental factors. Genetic factors, such as having a family history of depression, can make a person more prone to developing depression. Environmental factors, such as stressors like the loss of a loved one, family conflicts, or financial problems, can also contribute to the development of depression. It's important to remember that there is no single cause of depression, and everyone's experience is unique. If you're feeling persistently sad or experiencing a loss of interest in activities, it may be helpful to speak with a mental health professional for further evaluation and support.
Human: What are the symptoms of depression?
AI: Based on our previous conversation, the symptoms of depression include:

1. Depressed, sad, or irritable mood
2. Significant loss of interest or pleasure in activities
3. Significant weigh

In [ ]:
# Implement Classification
